# Modelos de redes neuronales para estimar el riesgo de default de un cliente  (incumplimientos de pagos en creditos).




In [ ]:
Muriel Díaz Pérez
Ezequiel José Elguedo Santander
Rubén Hernández Ballestas
Héctor Julio Osorio

In [112]:
# Inicialmente se cargan las librerias para ciencia de datos
import pandas as pd
# Función para dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as m
# Funciones para generar una matriz de confusión y reporte con métricas para clasificadores
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import tensorflow as tf
import math
from sklearn.preprocessing import StandardScaler

# Clasificación

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
# Se realiza la lectura en Google Drive del archivo de los datos credit_card_default_processed.csv
#Riesgo de Default
#contexto: Este conjunto de datos contiene información de sobre incumplimientos de pagos en creditos
# y entrega información de los clientes de acuerdo a factores demográficos, datos crediticios, historial de pagos y estados de cuenta de tarjetas de crédito.
#Objetivo: Entrenar dos modelos de redes neuronales que permita estimar el riesgo de default de un cliente.

data = pd.read_csv('/content/drive/MyDrive/ACT4 tópicos/credit_card_default_processed.csv')
data.head()

,months_loan_duration,amount,installment_rate,residence_history,age,existing_credits,dependents,telephone,foreign_worker,default,...,property_other,property_real estate,property_unknown/none,installment_plan_bank,installment_plan_stores,housing_own,housing_rent,job_skilled employee,job_unemployed non-resident,job_unskilled resident
0,6,1169,4,4,67,2,1,1,1,0,...,0,1,0,0,0,1,0,1,0,0
1,48,5951,2,2,22,1,1,0,1,1,...,0,1,0,0,0,1,0,1,0,0
2,12,2096,2,3,49,1,2,0,1,0,...,0,1,0,0,0,1,0,0,0,1
3,42,7882,2,4,45,1,2,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,24,4870,3,4,53,2,2,0,1,1,...,0,0,1,0,0,0,0,1,0,0


In [117]:
# Se hace un proceso adicional sobre los nombres de las columnas
data.columns = data.columns.str.replace(r'<|>| |-|\/', '_')

<ipython-input-117-e7cc981b7608>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r'<|>| |-|\/', '_')


In [118]:
# Se asignan los valores de X y Y para generar los modelos
x = data.drop(columns=['default'])
y = data['default']

In [119]:
# Se divide el conjunto de datos para generar dos conjuntos, uno de prueba (x_test, y_test), equivalente al 30% de los datos
#y uno de entrenamiento (x_train, y_train) equivaklente al 70% de los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1010, stratify=y)
x_train.shape

(700, 48)

In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 49 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   months_loan_duration                   1000 non-null   int64
 1   amount                                 1000 non-null   int64
 2   installment_rate                       1000 non-null   int64
 3   residence_history                      1000 non-null   int64
 4   age                                    1000 non-null   int64
 5   existing_credits                       1000 non-null   int64
 6   dependents                             1000 non-null   int64
 7   telephone                              1000 non-null   int64
 8   foreign_worker                         1000 non-null   int64
 9   default                                1000 non-null   int64
 10  checking_balance___0_DM                1000 non-null   int64
 11  checking_balance___200_DM      

In [121]:
# Se realiza la Estandarizacion de los datos
standard = StandardScaler()
standard.fit(x_train)

StandardScaler()

In [122]:
x_train

,months_loan_duration,amount,installment_rate,residence_history,age,existing_credits,dependents,telephone,foreign_worker,checking_balance___0_DM,...,property_other,property_real_estate,property_unknown_none,installment_plan_bank,installment_plan_stores,housing_own,housing_rent,job_skilled_employee,job_unemployed_non_resident,job_unskilled_resident
114,12,1680,3,1,35,1,1,0,1,1,...,0,1,0,0,0,1,0,1,0,0
221,12,1200,4,4,23,1,1,1,1,1,...,0,0,0,1,0,0,1,1,0,0
530,36,2273,3,1,32,2,2,0,1,0,...,1,0,0,0,0,1,0,1,0,0
626,6,2116,2,2,41,1,1,1,1,0,...,0,1,0,0,0,1,0,1,0,0
425,18,2779,1,3,21,1,1,1,1,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,24,4110,3,4,23,2,2,0,1,1,...,0,0,1,1,0,0,1,1,0,0
739,30,4280,4,4,26,2,1,0,1,0,...,1,0,0,0,0,0,1,0,0,1
776,36,3535,4,4,37,2,1,1,1,0,...,1,0,0,0,0,1,0,1,0,0
364,18,2473,4,1,25,1,1,0,1,1,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
x_train.values

array([[  12, 1680,    3, ...,    1,    0,    0],
       [  12, 1200,    4, ...,    1,    0,    0],
       [  36, 2273,    3, ...,    1,    0,    0],
       ...,
       [  36, 3535,    4, ...,    1,    0,    0],
       [  18, 2473,    4, ...,    0,    1,    0],
       [  30, 4249,    4, ...,    0,    0,    0]])

In [123]:
# Se tranforman los datos
x_train_esc = standard.transform(x_train)
x_test_esc = standard.transform(x_test)

In [125]:
x_train_esc

array([[-0.75181902, -0.56424061, -0.00781555, ...,  0.76635604,
        -0.15294382, -0.5       ],
       [-0.75181902, -0.73177181,  0.90399841, ...,  0.76635604,
        -0.15294382, -0.5       ],
       [ 1.25653836, -0.35726977, -0.00781555, ...,  0.76635604,
        -0.15294382, -0.5       ],
       ...,
       [ 1.25653836,  0.08319769,  0.90399841, ...,  0.76635604,
        -0.15294382, -0.5       ],
       [-0.24972968, -0.2874651 ,  0.90399841, ..., -1.30487651,
         6.53834842, -0.5       ],
       [ 0.75444901,  0.33240036,  0.90399841, ..., -1.30487651,
        -0.15294382, -0.5       ]])

## Multilayer perceptron

Para este modelo tenemos los siguientes parámetros:

* **hidden_layer_sizes:** Es una tupla que representa la cantidad de neuronas en cada capa. Ej: (8,4), significa 2 capas, la primera con 8 neuronas y la segunda con 4 neuronas.
* **activation: ** Es la función de activación para las capas ocultas. Una unidad lineal rectificada (ReLU) es una función de activación que introduce la propiedad de la no linealidad en un modelo de aprendizaje profundo y resuelve el problema de los gradientes de desaparición. Esta es la razón por la que es tan popular.
* **solver:** El optimizador que va a determinar los pesos en cada neurona.
El algoritmo de optimización de Adam es una extensión del descenso de gradiente estocástico que recientemente ha visto una adopción más amplia para aplicaciones de aprendizaje profundo en la visión por ordenador y el procesamiento del lenguaje natural.
* **alpha: ** Parámetro para la regularización.
* **batch_size:** Cantidad de registros utilizados en las iteraciones de la optimización.
* **learning_rate:** Se utiliza en los optimizadores adam y sgd, se utiliza para controlar la velocidad con la que se ajustan los pesos en cada iteración.
* **max_iter: **Cantidad de iteraciones máximas para llegar al óptimo.

In [ ]:
# Inicializar modelo (MODELO 1 MLPCLASSIFIER)
mlp = MLPClassifier(hidden_layer_sizes=(8,4), activation='relu', solver='adam', max_iter=100)

In [ ]:
# Entrenar modelo
mlp.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(8, 4), max_iter=100)

In [126]:
# Predicciones
y_pred_train = mlp.predict_proba(x_train_esc)[:,1] > 0.3
y_pred_test = mlp.predict_proba(x_test_esc)[:,1] > 0.3

In [127]:
# # Métricas del modelo (MODELO 1 MLPCLASSIFIER)
print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.94      0.80      0.86       490
           1       0.65      0.89      0.75       210

    accuracy                           0.82       700
   macro avg       0.80      0.84      0.81       700
weighted avg       0.85      0.82      0.83       700

              precision    recall  f1-score   support

           0       0.84      0.75      0.79       210
           1       0.53      0.67      0.59        90

    accuracy                           0.72       300
   macro avg       0.69      0.71      0.69       300
weighted avg       0.75      0.72      0.73       300



In [128]:
# Inicializar segundo modelo (variacion del parametro hidden_layer_sizes) (MODELO 2 MLPCLASSIFIER)
mlp = MLPClassifier(hidden_layer_sizes=(12,4), activation='relu', solver='adam', max_iter=100)

In [129]:
# Entrenar modelo
mlp.fit(x_train_esc, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(12, 4), max_iter=100)

In [ ]:
# Predicciones
y_pred_train = mlp.predict_proba(x_train_esc)[:,1] > 0.3
y_pred_test = mlp.predict_proba(x_test_esc)[:,1] > 0.3

In [130]:
# Métricas del modelo (MODELO 2 MLPCLASSIFIER)
print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.94      0.80      0.86       490
           1       0.65      0.89      0.75       210

    accuracy                           0.82       700
   macro avg       0.80      0.84      0.81       700
weighted avg       0.85      0.82      0.83       700

              precision    recall  f1-score   support

           0       0.84      0.75      0.79       210
           1       0.53      0.67      0.59        90

    accuracy                           0.72       300
   macro avg       0.69      0.71      0.69       300
weighted avg       0.75      0.72      0.73       300



## Tensorflow

TensorFlow facilita la creación de modelos de aprendizaje automático.

Para crear redes neuronales en tensorflow necesitamos:

* Definir cada una de las capas [Dense]
* En cada capa definir las neuronas y las funciones de activación de cada capa.
* Definir el optimizador y otros parámetros [compile]
* Ajustar el modelo [fit]
* Evaluar modelo [evaluate]


In [ ]:
x_train.shape

(700, 48)

In [ ]:
x_train.shape[0]

700

In [ ]:
x_train.shape[1]

48

In [132]:
# Crear capas
# tf.keras.models.Sequential tf.keras es la API de alto nivel de TensorFlow para construir y entrenar modelos de aprendizaje profundo
# Tener presente el input_shape, usar capas densas
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(24,activation='relu',input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(4, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [133]:
# Compilar modelo (MODELO 3 TENSORFLOW)
# model.compile

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
# Ajustar modelo
# model.fit
# epochs
# validation_split
# batch_size
model.fit(x_train, y_train, epochs=15, validation_split=0.1)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 1s 15ms/step - loss: 0.7505 - accuracy: 0.5524 - val_loss: 0.6405 - val_accuracy: 0.7429
Epoch 2/15
 1/20 [>.............................] - ETA: 0s - loss: 0.6387 - accuracy: 0.7500

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 0s 4ms/step - loss: 0.6509 - accuracy: 0.6952 - val_loss: 0.6361 - val_accuracy: 0.7429
Epoch 3/15
20/20 [==============================] - 0s 5ms/step - loss: 0.6477 - accuracy: 0.6952 - val_loss: 0.6316 - val_accuracy: 0.7429
Epoch 4/15
20/20 [==============================] - 0s 5ms/step - loss: 0.6445 - accuracy: 0.6952 - val_loss: 0.6273 - val_accuracy: 0.7429
Epoch 5/15
20/20 [==============================] - 0s 5ms/step - loss: 0.6416 - accuracy: 0.6952 - val_loss: 0.6231 - val_accuracy: 0.7429
Epoch 6/15
20/20 [==============================] - 0s 4ms/step - loss: 0.6388 - accuracy: 0.6952 - val_loss: 0.6194 - val_accuracy: 0.7429
Epoch 7/15
20/20 [==============================] - 0s 4ms/step - loss: 0.6362 - accuracy: 0.6952 - val_loss: 0.6159 - val_accuracy: 0.7429
Epoch 8/15
20/20 [==============================] - 0s 4ms/step - loss: 0.6339 - accuracy: 0.6952 - val_loss: 0.6124 - val_accuracy: 0.7429
Epoch 9/15
20/20 [=============

In [ ]:
# Evaluar resultados
# model.evaluate
# batch_size
model.evaluate(x_train_esc, y_train)

22/22 [==============================] - 0s 2ms/step - loss: 0.8219 - accuracy: 0.3243


[0.8219043016433716, 0.32428571581840515]

In [ ]:
model.evaluate(x_test_esc, y_test)

10/10 [==============================] - 0s 3ms/step - loss: 0.8116 - accuracy: 0.3167


[0.8115822672843933, 0.3166666626930237]

In [ ]:
# Generar predicciones
# model.predict
y_pred_train = model.predict(x_train_esc).reshape(1,-1)[0]
y_pred_test = model.predict(x_test_esc).reshape(1,-1)[0]

10/10 [==============================] - 0s 2ms/step


In [124]:
#METRICAS (MODELO 3 TENSORFLOW)
print(classification_report(y_train, y_pred_train > 0.3))
print(classification_report(y_test, y_pred_test > 0.3))

              precision    recall  f1-score   support

           0       0.91      0.68      0.78       490
           1       0.53      0.84      0.65       210

    accuracy                           0.73       700
   macro avg       0.72      0.76      0.71       700
weighted avg       0.79      0.73      0.74       700

              precision    recall  f1-score   support

           0       0.73      0.57      0.64       210
           1       0.34      0.51      0.41        90

    accuracy                           0.55       300
   macro avg       0.53      0.54      0.52       300
weighted avg       0.61      0.55      0.57       300

